# 3장. 사전학습 모델에 대한 미세조정
- [강좌링크](https://wikidocs.net/166800)

기존 pretrained model의 fine-tuning 방법
- Hub에서 대규모 데이터셋 가져오기
- 고급 Trainer API를 사용해 Fine-tuning
- Custom Training Loop
- &#129303;Accelerate 라이브러리를 활용하여 분산 환경에서 custom training loop를 쉽게 실행

# 1. 데이터 처리 작업

2장에서 공부한 것과 같이, PyTorch에서 단일 batch 기반으로 sequence classifier를 학습하는 방법은 다음과 같다.

In [4]:
import torch

from custom_utils import *

from accelerate import Accelerator, notebook_launcher
from datasets import load_dataset, load_metric
from transformers import (AdamW,
                          AutoTokenizer,
                          AutoModelForSequenceClassification,
                          DataCollatorWithPadding,
                          TrainingArguments,
                          Trainer,
                          get_scheduler)
from tqdm.auto import tqdm

import numpy as np
from torch.utils.data import DataLoader

wrapper = CustomObject()
wrapper.checkpoint = "bert-base-uncased"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.checkpoint)
wrapper.model = AutoModelForSequenceClassification.from_pretrained(wrapper.checkpoint)
wrapper.sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!"
]
wrapper.batch = wrapper.tokenizer(wrapper.sequences, padding = True, truncation = True, return_tensors = "pt")

wrapper.batch["labels"] = torch.tensor([1, 1]) # training을 위한 정답 label

wrapper.optimizer = AdamW(wrapper.model.parameters())
wrapper.loss = wrapper.model(**wrapper.batch).loss
wrapper.loss.backward() # 역전파
# wrapper.optimizer.zero_grad(): PyTorch는 기본적으로 RNN지원 위해 gradient를 누적함 따라서 업데이트 전 0으로 바꿔줘야 함.
wrapper.optimizer.step() # parameter update

wrapper.init_wrapper()


GPU NVIDIA GeForce RTX 3060
memory occupied: 2.10GB
Allocated GPU Memory: 0.41GB
Reserved GPU Memory: 0.46GB


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



GPU NVIDIA GeForce RTX 3060
memory occupied: 2.10GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


이 섹션에서는 William B.Dolan과 Chris Brockett의 논문에서 소개된 MRPC(Microsoft Research Paraphrase Corpus) 데이터셋을 예제로 사용한다.

이 데이터셋은 5,801건의 문장 쌍으로 구성되어 있으며 각 문장 쌍의 관계가 의역인지 여부를 판단하는 레이블이 존재함.

## 허브에서 데이터셋 로딩

MRPC 데이터셋은 10개의 데이터셋으로 구성된 GLUE 벤치마크 중 하나이다.
> GLUE 벤치마크: 10가지 텍스트 분류 작업을 통해 기계학습 모델의 성능을 측정하기 위한 학술적 벤치마크 데이터 집합

&#129303;Datasets 라이브러리는 허브에서 데이터셋을 다운로드하고 캐시할 수 있는 쉬운 명령어를 제공한다.

In [24]:
wrapper.raw_datasets = load_dataset("glue", "mrpc")
wrapper.raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [25]:
wrapper.raw_train_dataset = wrapper.raw_datasets["train"]
print(wrapper.raw_train_dataset[0])
print(wrapper.raw_train_dataset.features)

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None), 'idx': Value(dtype='int32', id=None)}


## Preprocess Dataset

다음과 같이 각 쌍의 모든 첫 번째 문장과 두 번째 문장을 각각 직접 토큰화할 수 있다.

In [26]:
wrapper.checkpoint = "bert-base-uncased"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.checkpoint)

wrapper.tokenized_dataset = wrapper.tokenizer(
    wrapper.raw_train_dataset["sentence1"],
    wrapper.raw_train_dataset["sentence2"],
    padding = True,
    truncation = True
)

위 방법은 sentence1, sentence2 쌍의 dataset을 만들지만 데이터가 담겨진 다차원 리스트가 키로 지정된 tokenized_dataset이라는 별도의 Python Dictionary를 반환하는 단점이 있으며, 토큰화하는 동안 전체 데이터셋을 저장할 충분한 공간의 RAM이 있는 경우에만 작동한다.

특정 데이터를 `dataset` 객체로 유지하기 위해 `Dataset.map()` 메서드를 사용한다. Python의 `map()`과 같은 lambda 방식으로 작동하며 dataset 객체 요소마다 실행할 메서드를 정의하고 다음과 같이 사용한다.

In [27]:
def tokenize_function(example):
    return wrapper.tokenizer(example["sentence1"], example["sentence2"], truncation = True)

"""
한 번에 모든 데이터셋에 토큰화 기능 적용
"""
wrapper.tokenized_datasets = wrapper.raw_datasets.map(tokenize_function, batched = True)
print(wrapper.tokenized_datasets)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


&#129303;Datasets 라이브러리는 데이터셋에 새로운 필드들을 추가하는데 이는 preprocess 함수에서 반환된 사전의 각 키에 해당한다.
- input_ids
- token_type_ids
- attention_mask

`num_proc` 매개변수를 전달하여 `map()`으로 전처리 기능을 적용할 때 multi-processing을 사용할 수도 있다.

&#129303;Tokenizers 라이브러리는 샘플을 더 빠르게 토큰화하기 위해 이미 다중 스레드를 사용하지만 이 라이브러리에서 지원하는 fast tokenizer를 사용하지 않는 경우라면 속도가 더 빨라질 수 있다.

전처리 함수가 `map()`을 적용한 데이터셋의 기존 키들(idx, label) 등에 대한 새로운 값을 반환한 경우 기존 필드(idx, label, sentence1, sentence2 등)들을 변경할 수도 있다.

마지막으로 할 일은 전체 요소들을 batch로 분리할 때 가장 긴 요소의 길이로 모든 예제를 채우는 것인데 이를 동적 패딩이라고 한다.

## Dynamic Padding

샘플들을 함께 모아 지정된 크기의 batch로 구성하는 `collate function`은 `DataLoader`를 build할 때 매개변수로 전달할 수 있다.

기본값은 단순히 샘플들을 PyTorch 텐서로 변환하고 결합하는 함수이다.

위 코드들은 패딩 작업을 미뤄와서 입력값이 모두 동일한 크기가 아닌데 그 이유는 전체 데이터셋이 아닌 개별 batch에 대해 별도로 padding 작업을 수행하여 과도하게 긴 입력으로 인한 과도한 padding 작업을 방지하기 위함이나 이렇게 하면 학습 속도가 빨라지지만 TPU에서 학습하는 경우 문제가 발생할 수 있다. TPU는 추가적인 padding이 필요한 경우에도 전체 데이터셋이 고정된 형태를 선호한다.

batch로 분리하려는 데이터셋의 요소 각각에 대해 정확한 수의 padding을 적용할 수 있는 collate function을 정의해야 하는데 &#129303;Transformers 라이브러리는 `DataCollatorWithPadding`을 통해 이런 기능을 제공한다. 이 함수는 Tokenizer를 입력으로 받아 padding 토큰이 무엇인지와 왼쪽과 오른쪽 중 어느 쪽에 padding을 수행할지를 파악한다.

In [28]:
wrapper.data_collator = DataCollatorWithPadding(tokenizer = wrapper.tokenizer)

In [29]:
wrapper.samples = wrapper.tokenized_datasets["train"][:8]
wrapper.samples = {k: v for k, v in wrapper.samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in wrapper.samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [30]:
wrapper.batch = wrapper.data_collator(wrapper.samples)
print({k: v.shape for k, v in wrapper.batch.items()})

del tokenize_function
wrapper.init_wrapper()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]), 'token_type_ids': torch.Size([8, 67]), 'attention_mask': torch.Size([8, 67]), 'labels': torch.Size([8])}

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.07GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 2. Trainer API를 이용한 Fine-tuning

&#129303;Transformers 라이브러리는 `Trainer` 클래스를 제공해 fine-tuning을 지원한다.

위의 모든 예제를 아래 코드로 요약 후 실행했다고 가정한 후 실습함.

In [31]:
wrapper.raw_datasets = load_dataset("glue", "mrpc")
wrapper.checkpoint = "bert-base-uncased"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.checkpoint)

def tokenize_function(example):
    return wrapper.tokenizer(example["sentence1"], example["sentence2"], truncation = True)

wrapper.tokenized_datasets = wrapper.raw_datasets.map(tokenize_function, batched =  True)
wrapper.data_collator = DataCollatorWithPadding(tokenizer = wrapper.tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

## Training

Fine-tuning을 위한 하이퍼 파라미터 `TrainingArguments` 클래스 정의 및 모델(`AutoModelForSequenceClassification`) 정의

In [32]:
wrapper.training_arguments = TrainingArguments("test-trainer")
wrapper.model = AutoModelForSequenceClassification.from_pretrained(wrapper.checkpoint, num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


위에서 정의한 모델, 하이퍼 파라미터, 데이터 셋과 collate function, tokenizer 등의 개체를 전달하여 `Trainer`를 정의하고 `train()`으로 학습을 진행한다.

In [34]:
wrapper.trainer = Trainer(
    wrapper.model,
    wrapper.training_arguments,
    train_dataset = wrapper.tokenized_datasets["train"],
    eval_dataset = wrapper.tokenized_datasets["validation"],
    data_collator = wrapper.data_collator,
    tokenizer = wrapper.tokenizer
)

wrapper.trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=1377, training_loss=0.34335581184558966, metrics={'train_runtime': 136.4452, 'train_samples_per_second': 80.648, 'train_steps_per_second': 10.092, 'total_flos': 405324636337200.0, 'train_loss': 0.34335581184558966, 'epoch': 3.0})

fine-tuning이 시작되고 500단계마다 traing loss가 보고되지만 모델 성능이 좋은지 나쁜지는 알려주지 않는 이유는 아래와 같다.
1. 학습 과정에서 평가가 수행되도록 trainer에게 `evaluation_strategy` 매개변수를 "steps" 또는 "epochs"로 지정하지 않았다.
2. 평가 방법 혹은 평가 척도를 정의한 `compute_metrics()` 함수를 지정하지 않았다.

## Evaluation

유용한 compute_metrics() 함수를 구현하고 이를 사용하는 방법은 `EvalPrediction` 객체(__predictions__ 필드와 __label_ids__ 필드가 포함된 named tuple)를 필요로 하며 문자열과 실수값을 매핑하는 dictionary를 반환한다. 문자열은 metrics의 이름이고 실수값은 해당 metric에 기반한 평가 결과값이다.

예측 결과를 얻으려면 `Trainer.predict()` 명령을 사용한다.

In [36]:
wrapper.predictions = wrapper.trainer.predict(wrapper.tokenized_datasets["validation"])
print(wrapper.predictions.predictions.shape, wrapper.predictions.label_ids.shape)

(408, 2) (408,)


408은 예측한 데이터의 개수, 데이터셋의 각 요소에 대한 logit값이다. 이 logit값들을 레이블과 비교할 수 있는 예측 결과로 변환하려면 second axis에 존재하는 항목에서 최대값이 있는 인덱스를 가져와야 한다.

In [37]:
wrapper.preds = np.argmax(wrapper.predictions.predictions, axis = -1)

이제 `preds`를 레이블과 비교할 수 있다. `datasets` 라이브러리의 `load_metric()` 함수를 이용해 MRPC 데이터셋과 관련된 평가지포(metric)를 로드하고 `compute_metrics()` 함수를 얻을 수 있다. 각 epoch이 끝날 때 metric을 출력하기 위해 compute_metrics() 함수를 사용해 새 Trainer를 정의하고 학습을 진행하면 학습 손실 외에 각 epoch이 끝날 때 validation loss 및 metrics를 보고한다.

In [39]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis = -1)
    return metric.compute(predictions = predictions, references = labels)


wrapper.training_arguments = TrainingArguments("test-trainer-with-metric", evaluation_strategy = "epoch")
wrapper.model = AutoModelForSequenceClassification.from_pretrained(wrapper.checkpoint, num_labels = 2)

wrapper.trainer = Trainer(
    wrapper.model,
    wrapper.training_arguments,
    train_dataset = wrapper.tokenized_datasets["train"],
    eval_dataset = wrapper.tokenized_datasets["validation"],
    data_collator = wrapper.data_collator,
    tokenizer = wrapper.tokenizer,
    compute_metrics = compute_metrics
)

wrapper.trainer.train()

del compute_metrics
wrapper.init_wrapper()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/jongg/.virtualenvs/HF_Transformers/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss



GPU NVIDIA GeForce RTX 3060
memory occupied: 4.27GB
Allocated GPU Memory: 0.02GB
Reserved GPU Memory: 0.04GB


# 3. Full Training

데이터 처리를 완료했다고 가정 하에 Trainer 클래스 없이 위와 동일한 결과를 얻는 방법

In [40]:
wrapper.raw_datasets = load_dataset("glue", "mrpc")
wrapper.checkpoint = "bert-base-uncased"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.checkpoint)

def tokenize_function(example):
    return wrapper.tokenizer(example["sentence1"], example["sentence2"], truncation = True)

wrapper.tokenized_datasets = wrapper.raw_datasets.map(tokenize_function, batched = True)
wrapper.data_collator = DataCollatorWithPadding(tokenizer = wrapper.tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

## 학습을 위한 준비

실제 training loop를 정의하기 전에 몇 가지 객체를 정의해야 한다.
1. batch를 반복하는데 사용할 `dataloaders`
    - Trainer가 자동으로 수행한 몇 가지 작업을 직접 처리하기 위해 tokenized_datasets에 약간의 후처리 적용 필요
      1. 모델이 필요로하지 않는 값이 저장된 columns 제거(sentence1, sentence2 등)
      2. column label의 이름을 labels로 변경
      3. 파이썬 리스트 대신 PyTorch tensor를 반환하도록 datasets 형식 지정
2. Optimizer
3. Learning rate scheduler

In [41]:
wrapper.tokenized_datasets = wrapper.tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
wrapper.tokenized_datasets = wrapper.tokenized_datasets.rename_column("label", "labels")
wrapper.tokenized_datasets.set_format("torch")
print(wrapper.tokenized_datasets["train"].column_names)

wrapper.train_dataloader = DataLoader(
    wrapper.tokenized_datasets["train"],
    shuffle = True,
    batch_size = 8,
    collate_fn = wrapper.data_collator # batch 내에서의 최대 길이로 padding
)

wrapper.eval_dataloader = DataLoader(
    wrapper.tokenized_datasets["validation"],
    batch_size = 8,
    collate_fn = wrapper.data_collator
)

# 데이터 처리에 오류가 있는지 확인하기 위해 batch 검사
for batch in wrapper.train_dataloader:
    print({k: v.shape for k, v in batch.items()})
    break

# model instantiate
wrapper.model = AutoModelForSequenceClassification.from_pretrained(wrapper.checkpoint, num_labels = 2)


# Optimizer AdamW, Adam with weight decay regularization
wrapper.optimizer = AdamW(wrapper.model.parameters(), lr = 5e-5)


# Learning Rate Scheduler
"""
default의 경우 최대값 5e-5에서 0까지 선형 감쇠한다.
이를 적절하게 정의하려면 우리가 수행할 학습 단계의 횟수를 알아야 하는데 이는 epoch * batch 개수(DataLoader의 길이)의 값과 같다.
"""

wrapper.num_epochs = 3
wrapper.num_training_steps = wrapper.num_epochs * len(wrapper.train_dataloader)
wrapper.lr_scheduler = get_scheduler(
    "linear",
    optimizer = wrapper.optimizer,
    num_warmup_steps = 0,
    num_training_steps = wrapper.num_training_steps
)
print(wrapper.num_training_steps)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['labels', 'input_ids', 'token_type_ids', 'attention_mask']
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 75]), 'token_type_ids': torch.Size([8, 75]), 'attention_mask': torch.Size([8, 75])}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1377


/home/jongg/.virtualenvs/HF_Transformers/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Training Loop

GPU로 모델 이동 후 학습

학습이 언제 끝날 지 정보를 얻기 위해 tqdm 라이브러리를 사용하여 training steps를 기준으로 progress bar를 출력할 수 있도록 한다.

In [42]:
wrapper.model.to("cuda")

wrapper.progress_bar = tqdm(range(wrapper.num_training_steps))

wrapper.model.train()
for epoch in range(wrapper.num_epochs):
    for batch in wrapper.train_dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}
        outputs = wrapper.model(**batch)
        loss = outputs.loss
        loss.backward() # 손실값 역전파
        
        wrapper.optimizer.step() # gradient 적용
        wrapper.lr_scheduler.step()
        wrapper.optimizer.zero_grad() # gradient 누적 방지 초기화
        wrapper.progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

## Evaluation Loop

위에서처럼 &#129303;Datasets 라이브러리에서 제공하는 평가 메트릭을 사용한다.

이미 metric.compute() 메서드를 살펴보았지만, `metric.add_batch()` 메서드로 평가 루프를 실행하면서 batch별 평가 metric 결과를 누적할 수 있다.

모든 batch를 누적하고 나면 그때 `metric.compute()`로 최종 결과를 얻을 수 있다.

In [44]:
wrapper.metric = load_metric("glue", "mrpc")
wrapper.model.eval()

for batch in wrapper.eval_dataloader:
    batch = {k: v.to("cuda") for k, v in batch.items()}
    with torch.no_grad():
        outputs = wrapper.model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim = -1)
    wrapper.metric.add_batch(predictions = predictions, references = batch["labels"])

wrapper.metric.compute()

wrapper.init_wrapper()


GPU NVIDIA GeForce RTX 3060
memory occupied: 4.28GB
Allocated GPU Memory: 0.43GB
Reserved GPU Memory: 0.46GB


## &#129303;Accelerate 라이브러리를 사용한 가속화

accelerator는 windows에서 안되는걸로 알고 [wsl에 ubuntu, cuda toolkit, cudnn 설치](https://www.notion.so/jonggurl96/WSL-Ubuntu-20-04-CUDA-cudnn-965f97ad8a1f4fd4b5c1b03742b8cd15?pvs=4) 후 사용

터미널에서 `accelerate config` 명령어로 다음과 같이 설정함: [default_config.yaml](../assets/default_config.yaml)

분산 학습 시작 명령어는 `accelerate launch train.py`와 같이 파이썬파일을 명령어로 실행해야 하고 Notebook에서는 다음과 같다.

```python
def training_function():
    pass

from accelerate import notebook_launcher

notebook_launcher(training_function)
```

#### Training Loop with Accelerate

In [7]:
wrapper.checkpoint = "bert-base-uncased"

# Tokenizer
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.checkpoint)

# Tokenize Function
wrapper.raw_datasets = load_dataset("glue", "mrpc")
def tokenize_function(example):
    return wrapper.tokenizer(example["sentence1"], example["sentence2"], truncation = True)

wrapper.tokenized_datasets = wrapper.raw_datasets.map(tokenize_function, batched = True)
wrapper.tokenized_datasets = wrapper.tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
wrapper.tokenized_datasets = wrapper.tokenized_datasets.rename_column("label", "labels")
wrapper.tokenized_datasets.set_format("torch")

# Collate Function
wrapper.data_collator = DataCollatorWithPadding(tokenizer = wrapper.tokenizer)

# Training DataLoader
wrapper.train_dataloader = DataLoader(
    wrapper.tokenized_datasets["train"],
    shuffle = True,
    batch_size = 8,
    collate_fn = wrapper.data_collator
)

# Validation DataLoader
wrapper.eval_dataloader = DataLoader(
    wrapper.tokenized_datasets["validation"],
    batch_size = 8,
    collate_fn = wrapper.data_collator
)

# Model
wrapper.model = AutoModelForSequenceClassification.from_pretrained(wrapper.checkpoint, num_labels = 2)

# Optimizer
wrapper.optimizer = AdamW(wrapper.model.parameters(), lr = 3e-5)

# accelerate
wrapper.accelerator = Accelerator()
wrapper.train_dataloader, wrapper.eval_dataloader, wrapper.model, wrapper.optimizer = wrapper.accelerator.prepare(
    wrapper.train_dataloader, wrapper.eval_dataloader, wrapper.model, wrapper.optimizer
)

wrapper.num_epochs = 3
wrapper.num_training_steps = wrapper.num_epochs * len(wrapper.train_dataloader)
wrapper.lr_scheduler = get_scheduler(
    "linear",
    optimizer = wrapper.optimizer,
    num_warmup_steps = 0,
    num_training_steps = wrapper.num_training_steps
)

progress_bar = tqdm(range(wrapper.num_training_steps))

def training_fn():
    wrapper.model.train()
    
    for epoch in range(wrapper.num_epochs):
        for batch in wrapper.train_dataloader:
            batch = {k: v.to("cuda") for k, v in batch.items()}
            outputs = wrapper.model(**batch)
            loss = outputs.loss
            wrapper.accelerator.backward(loss)
            
            wrapper.optimizer.step()
            wrapper.lr_scheduler.step()
            wrapper.optimizer.zero_grad()
            progress_bar.update(1)


notebook_launcher(training_fn, num_processes = 1)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Launching training on one GPU.


#### Evaluation Loop with Accelerate

In [9]:
# 평가 메트릭
wrapper.metric= load_metric("glue", "mrpc")

def eval_fn():
    wrapper.model.eval()
    for batch in wrapper.eval_dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = wrapper.model(**batch)
        
        logits = outputs.logits
        predictions = torch.argmax(logits, dim = -1)
        wrapper.metric.add_batch(predictions = predictions, references = batch["labels"])
    
    # 평가 결과 계산 및 출력
    print(wrapper.metric.compute())
    
notebook_launcher(eval_fn, num_processes = 1)

wrapper.init_wrapper()

Launching training on one GPU.
{'accuracy': 0.8627450980392157, 'f1': 0.902439024390244}

GPU NVIDIA GeForce RTX 3060
memory occupied: 4.57GB
Allocated GPU Memory: 0.02GB
Reserved GPU Memory: 0.04GB
